# Query Open-Meteo weather data for trip locations
This notebook demonstrates how to fetch hourly weather data for each trip in `cleaned_trip_data.csv`.
The start times are rounded to the nearest hour before performing the request.

In [ ]:
import pandas as pd
import requests_cache
from retry_requests import retry
import openmeteo_requests

# Setup caching and retry behaviour
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

In [ ]:
# Load cleaned trip data
df = pd.read_csv('jupyter/cleaned_trip_data.csv', parse_dates=['start_time'])

# Round start_time to the nearest hour
df['time_rounded'] = df['start_time'].dt.round('h')

# Display the rounded time with hour information
df_display = df.copy()
df_display['time_rounded'] = df_display['time_rounded'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_display.head()

In [ ]:
def fetch_hourly_weather(lat, lon, timestamp):
    """Request hourly weather data for a specific time and location.
    The timestamp is expected in UTC.
    """
    url = 'https://api.open-meteo.com/v1/forecast'
    params = {
        'latitude': lat,
        'longitude': lon,
        'hourly': ['temperature_2m', 'precipitation'],
        'start': int(timestamp.timestamp()),
        'timezone': 'UTC',
        'past_days': 0
    }
    
    responses = openmeteo.weather_api(url, params=params)
    response = responses[0]
    hourly = response.Hourly()
    temperature = hourly.Variables(0).ValuesAsNumpy()[0]
    precipitation = hourly.Variables(1).ValuesAsNumpy()[0]
    return temperature, precipitation

In [ ]:
# Example: fetch weather for the first trip
row = df.iloc[0]
temperature, precipitation = fetch_hourly_weather(row['start_lat'], row['start_lon'], row['time_rounded'])
temperature, precipitation